# Case Studies:
- VGG-16 (from Keras models repository, fine tuned)
- ResNet152
- XCeption

> https://www.topbots.com/important-cnn-architectures/

In [1]:
# https://keras.io/guides/transfer_learning/

# Xception
Before training and fine tuning lets see the performance of the pretrained model on some samples.

In [ ]:
import keras
untouched_xception = keras.applications.Xception(weights='imagenet')
# Try on 10 different images
untouched_xception.predict()

In [ ]:
from models.structure.default_augmented_model_family import ChannelsLastFixAugmentedNaiveDNNModelFamily

# With augmentation
class XceptionAugmented(ChannelsLastFixAugmentedNaiveDNNModelFamily):
    def make_layers(self, input_shape: (int, int, int)) -> tuple[keras.Layer, keras.Layer]:
        inputs = keras.Input(input_shape)
        x = keras.applications.xception.preprocess_input(inputs, data_format="channels_first")
        x = keras.applications.Xception(weights='imagenet', include_top=False)(x, training=False, pooling='max')
        outputs = keras.layers.Dense(1, activation='sigmoid')(x)

        return inputs, outputs

In [ ]:
from torch.utils.data import DataLoader
from dataset.k_fold_dataset_wrapper import KFoldDatasetWrapper

from dataset.dataset_loader import dataset_loader

train, test = dataset_loader((299, 299), is_grayscale=False)
dataset_split_controller = KFoldDatasetWrapper(5)
dataset_split_controller.load_data(train)

local_train, validation = dataset_split_controller.get_data_for_fold(0)
train_dataloader = DataLoader(dataset=local_train, batch_size=16, shuffle=True)
validation_dataloader = DataLoader(dataset=validation, batch_size=16, shuffle=True)

In [ ]:
model = XceptionAugmented("AugmentedXCeption", "binary_crossentropy").make_model((3, 299, 299))
model.compile(optimizer=keras.optimizers.Adam(),
              loss=keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=[keras.metrics.BinaryAccuracy()])

model.summary()

In [ ]:
history = model.fit(local_train, epochs=20, validation_data=validation_dataloader)

# VGG-16
https://arxiv.org/abs/1409.1556

# Resnet-152